In [1]:
# importing stuff
import time
import serial      # for communication with arduino
import collections # for ordered dicts (session_info)
import pysistence  # for immutable dict (mouse_info)
import warnings
import box_utils   # box_utils.py must be in the same folder as this file

In [2]:
# defining immutable mouse dict (once defined for a mouse, this should never change)
mouse_info = pysistence.make_dict({'mouseName': 'jaxmale08',
                 'requiredVersion': 7,
                 'leftVisCue': 0,
                 'rightVisCue': 3,
                 'leftAudCue': 3,
                 'rightAudCue': 0})

In [3]:
# Information for this session (the user should edit this each session)
# fix: insert date and time
session_info                              = collections.OrderedDict()
session_info['mouseName']                 = mouse_info['mouseName']
session_info['trainingPhase']             = 4
session_info['maxSessionLength_min']      = 60     # in minutes
session_info['maxTrials']                 = 1000   # program terminates when either maxSessionLength_min or maxTrials is reached
session_info['maxRewards']                = 200    # program also terminates if maxRewards is reached
session_info['interTrialInterval_mean']   = 0      # number of extra seconds between trials
session_info['interTrialInterval_SD']     = 0      # standard deviation of seconds between trials
session_info['punishForErrorPokeYN']      = 0      # 0 = no, 1 = yes for stage 5 only
session_info['cueWithdrawalPunishYN']     = 0      # only 1 in phase 4-5
session_info['initPokePunishYN']          = 0      # whether to punish for init poke between trials
session_info['nTrial']                    = [1];   # the number of the first trial
session_info['resetTimeYN']               = 1      # whether or not to restart arduino timer at beginning of session

# initializing trial L/R parameters
session_info['trialLRtype']          = [] # (1 = LX, 2 = XL, 3 = RX, 4 = XR, 5 = LR, 6 = RL).
session_info['trialLRtype_info']     = '(1 = LX, 2 = XL, 3 = RX, 4 = XR, 5 = LR, 6 = RL)'
session_info['LrewardCode']          = [] # will be set automatically later
session_info['RrewardCode']          = []

# this is planning for the future, when we will likely want two auditory
# stimuli and two visual stimuli. For now, just leave it as all 3's
session_info['trialAVtype']          = [3]  # 1 = auditory only, 2 = visual only, 3 = both aud + vis
session_info['trialAVtype_info']     = '1 = auditory only, 2 = visual only, 3 = both aud + vis'

# will use this later, when we do optogenetic manipulation
session_info['laserOnCode']          = [0]

# reward parameters for first trial
session_info['LrewardSize_nL']         = [5000] # the starting value, which will be updated over time
session_info['RrewardSize_nL']         = [5000]
session_info['rewardSizeMax_nL']       = [8000]
session_info['rewardSizeMin_nL']       = [2000]
session_info['rewardSizeDelta_nL']     = [500] # the number of nanoliters to adjust reward size by 
session_info['deliveryDuration_ms']    = 1000
session_info['syringeSize_mL']         = 5

# time intervals
session_info['readyToGoLength']        = 1000*30
session_info['punishDelayLength']      = 1000*6
session_info['goToPokesLength']        = 1000*60
session_info['rewardCollectionLength'] = 1000*5

# cue lengths, etc. - for phases 4 and 5, they are changed below
session_info['preCueLength']           = [0]
session_info['cue1Length']             = [100]
session_info['cue2Length']             = [100]
session_info['interOnsetInterval']     = [0]
session_info['postCueLength']          = [0]

# cue volume and brightness
session_info['WNvolume']               = 50     # volumes are 0-255
session_info['lowCueVolume']           = 120
session_info['highCueVolume']          = 120
session_info['buzzerVolume']           = 90
session_info['cueLED1Brightness']      = 1023   # brightness is 0-1023
session_info['cueLED2Brightness']      = 1023
session_info['cueLED3Brightness']      = 1023
session_info['cueLED4Brightness']      = 1023

# setting parameters based on training phase
if session_info['trainingPhase'] == 1:
	session_info['punishForErrorPokeYN']      = 0; # 0 = no, 1 = yes for stage 5 only
	session_info['cueWithdrawalPunishYN']     = 0; # only 1 in phase 4-5
	session_info['goToPokesLength']           = 60 * 1000;
elif session_info['trainingPhase'] == 2:
	session_info['punishForErrorPokeYN']      = 0; # 0 = no, 1 = yes for stage 5 only
	session_info['cueWithdrawalPunishYN']     = 0; # only 1 in phase 4-5
	session_info['goToPokesLength']           = 60 * 1000;
elif session_info['trainingPhase'] == 3:
	session_info['punishForErrorPokeYN']      = 0; # 0 = no, 1 = yes for stage 5 only
	session_info['cueWithdrawalPunishYN']     = 0; # only 1 in phase 4-5
	session_info['goToPokesLength']           = stage3_goToPokesLength; # fix
elif session_info['trainingPhase'] == 4:
	session_info['punishForErrorPokeYN']      = 0; # 0 = no, 1 = yes for stage 5 only
	session_info['cueWithdrawalPunishYN']     = 1; # only 1 in phase 4-5
	session_info['goToPokesLength']           = 4 * 1000;
elif session_info['trainingPhase'] == 5:
	session_info['punishForErrorPokeYN']      = 1; # 0 = no, 1 = yes for stage 5 only
	session_info['cueWithdrawalPunishYN']     = 1; # only 1 in phase 4-5
	session_info['goToPokesLength']           = 4 * 1000;
    
# initializing the cue/slot parameters
session_info['slot1_vis'] = []
session_info['slot1_aud'] = []
session_info['slot2_vis'] = []
session_info['slot2_aud'] = []
session_info['slot3_vis'] = []
session_info['slot3_aud'] = []
session_info['slot1Length'] = []
session_info['slot2Length'] = []
session_info['slot3Length'] = []

In [32]:
# figuring out time and date
import datetime

datetime.date()

TypeError: function missing required argument 'year' (pos 1)

In [4]:
# setting all parameters for the first trial

# set the trialLRtype for the first trial
# info for specific trials - need to update this later
# for stages 1-2, this should be [1 3]. For stage 3 and higher, it should be [1:6]
# i.e. no free choice until stage 3

session_info['trialLRtype'] = [1]  # fix: add function to pick them randomly

# set reward codes for the first trial
box_utils.make_reward_codes(session_info)

# set cue/slot cotes for the first trial
box_utils.make_cues(session_info, mouse_info)

In [5]:
# figure out the COM port
COM_port = 'COM5'  # fix: update this later

In [6]:
# make directory for logfile, add name of logfile to session_info, etc.


In [7]:
# making connection with arduino, checking that required version is correct
#connection_speed = 230400      # old value was 115200; let's see if we do 230400
connection_speed = 115200
try:
    del arduino
except:
    pass
arduino = serial.Serial(COM_port, connection_speed, timeout=10) # Establish the connection on a specific port
arduino.set_buffer_size(rx_size=10000000, tx_size=10000000)
time.sleep(1)  # required for connection to complete before transmitting
# arduino.write(b'calibrationLength;1000\n')  # for testing

In [8]:
# verify the arduino is running the correct version of the box code
arduino.flushInput()
arduino.write(bytes('checkVersion', 'utf-8'))
ver = int(arduino.readline())
if ver != mouse_info['requiredVersion']:
    raise Exception('This requires the arduino to run version ' + str(mouse_info['requiredVersion']) + \
                    ', but it is running ver ' + str(ver))
del ver

In [9]:
# initializing default box_params and sending to arduino
box_utils.set_box_defaults(arduino)

In [10]:
# create GUI to ask user to start trial


In [11]:
# start camera
box_utils.send_dict_to_arduino({'cameraRecordingYN' : 1}, arduino)

In [12]:
# loop over trials begins here

In [13]:
# if it's not the first trial, append stuff for the current trial here



In [14]:
# send session_info to arduino
box_utils.send_dict_to_arduino(session_info, arduino)

In [15]:
# start the trial
#arduino.write(bytes('trainingPhase;1', 'utf-8'))
# flush buffer
# arduino.flushInput()
arduino.write(bytes('startTrialYN;1', 'utf-8'))


14

In [16]:
# loop to log info from arduino while trial runs
Astr = str('')  # declare temp string
while not Astr.endswith('Standby;0'):
#x = 1
#while x < 5:    
   if arduino.in_waiting > 0:
      time.sleep(0.010); # to prevent readline() from being called before the entire string is written
      Astr = arduino.readline().decode('utf-8').rstrip()
      print(Astr)
      #fprintf(logfid, [Astr '\n']);
      #fprintf([Astr '\n']);
#      x += 1
   else:
      time.sleep(0.050)
del Astr

print('Trial completed')

544;1;TrialAvailable;0
544;1;TrainingPhase;4
544;1;requiredPokeHoldLength_ms;100
545;1;trialLRtype;1
545;1;trialAVtype;3
545;1;Lsize_nL;5000
545;1;Isize_nL;0
547;1;Rsize_nL;5000
3829;1;initPokeEntry;0
3829;1;TrialStarted_ms;3280
4099;1;initPokeExit_ms;270
4929;1;leftPokeEntry;0
4929;1;leftReward_nL;5000
4929;1;leftRewardCollected;0
4929;1;letTheAnimalDrink_ms;5000
5174;1;leftPokeExit_ms;245
8874;1;rightPokeEntry;0
9114;1;rightPokeExit_ms;240
9938;1;Standby;0
Trial completed


In [17]:
# evaluate whether or not to exit the loop

In [18]:
# optionally add random extra ITI

In [19]:
# end of loop

In [20]:
# stop camera
#arduino.write(bytes('cameraRecordingYN;0', 'utf-8'))
box_utils.send_dict_to_arduino({'cameraRecordingYN': 0}, arduino)

In [21]:
# save dicts to disk

In [22]:
# close arduino
time.sleep(1)
arduino.close()

In [23]:
import platform
platform.node()


'Luke-HP-laptop'

In [24]:
import socket
socket.gethostname()

'Luke-HP-laptop'

In [25]:
#del arduino